In [1]:
import numpy as np
import pandas as pd
import pyedflib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dropout, Flatten, Dense

In [2]:
def get_dataset(name):
    #Загружаем датасет
    def read_dataset(file_path):
        edf_file = pyedflib.EdfReader(file_path)
        n_signals = edf_file.signals_in_file
        signal_labels = edf_file.getSignalLabels()
        signals = [edf_file.readSignal(i) for i in range(n_signals)]
        edf_file.close()
        print('Сигналов обнаружено: ', n_signals)
        return signal_labels, signals

    def read_txt_markers(file_path):
        with open(file_path, 'r') as file:
            labels = file.read().splitlines()
        return labels
    
    def convert_to_sec(time: str):
        s = list(map(int, time.split(':')))
        return s[0]*3600 + s[1] * 60 + s[2]

    def get_markered_dataset(data_file_path, labels_file_path):
        signal_labels, signals = read_dataset(data_file_path)
        signals = np.array(signals)
        data = pd.DataFrame(signals).T.rename(columns={i: signal_labels[i] for i in range(len(signal_labels))})
        
        markers = read_txt_markers(labels_file_path)[1:]
        markers = [line.split('\t') for line in markers]
        markers = [[line[0], convert_to_sec(line[1])*400, line[2]] for line in markers]
        
        markers_df = pd.DataFrame(markers, columns=['id', 'time', 'marker'])
        markers_df['time'] = markers_df['time'].astype(int)
    
        data['target'] = None
    
        for i in range(0, len(markers_df), 2):
            start_marker = markers_df.iloc[i]
            end_marker = markers_df.iloc[i + 1]
            if start_marker['marker'].startswith('ds'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'ds'
            elif start_marker['marker'].startswith('is'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'is'
            elif start_marker['marker'].startswith('swd'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'swd'
        percentage_marked = data[data['target'].notna()].shape[0]/data.shape[0]
        print(f'Размечено {round(percentage_marked, 2)} данных')
        return data

    def load_marked_dataset(file, folder='ECoG_fully_marked_(4+2 files, 6 h each)', base_path=r"Downloads/train_dataset_minzdrav_train"):
        dataset_file_path = fr"{base_path}/{folder}/{file}.edf"
        markers_file_path = fr"{base_path}/{folder}/{file}.txt"
        dataset = get_markered_dataset(dataset_file_path, markers_file_path)
        return dataset

    dataset = load_marked_dataset(name)
    dataset = dataset[dataset['target'].notna()]
    return dataset

In [4]:
dataset_names = ['Ati4x1_15m_BL_6h', 
                 'Ati4x1_15m_Dex003(Pharm!)_6h', 
                 'Ati4x1_15m_H2O_6h', 
                 'Ati4x3_12m_BL_6h',
                 'Ati4x6_14m_BL_6h']
#'Ati4x3_9m_Xyl01(Pharm!)_6h', 
dataset_list = []
for dataset_name in dataset_names:
    dataset = get_dataset(dataset_name)
    dataset = dataset[dataset['target'].notna()]
    dataset_list.append(dataset)
    
all_datas = pd.concat(dataset_list, ignore_index=True)
all_datas

Сигналов обнаружено:  3
Размечено 0.42 данных
Сигналов обнаружено:  3
Размечено 0.25 данных
Сигналов обнаружено:  3
Размечено 0.22 данных
Сигналов обнаружено:  3
Размечено 0.23 данных
Сигналов обнаружено:  3
Размечено 0.34 данных


,FrL,FrR,OcR,target
0,-0.167625,-0.125625,-0.089688,ds
1,-0.175687,-0.100250,-0.038562,ds
2,-0.176687,-0.123250,-0.039438,ds
3,-0.174375,-0.127812,-0.014688,ds
4,-0.138375,-0.085625,0.032250,ds
...,...,...,...,...
12613995,-0.090625,0.000937,-0.033250,swd
12613996,-0.059188,0.032938,0.000438,swd
12613997,-0.056688,0.031000,-0.014563,swd
12613998,-0.047563,0.041250,-0.027938,swd


In [5]:
all_datas['target'] = all_datas['target'].replace({'ds': 0, 'is': 1, 'swd': 2})

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_8708\3848684522.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_datas['target'] = all_datas['target'].replace({'ds': 0, 'is': 1, 'swd': 2})


In [ ]:
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    
    # Сбрасываем индексы в y, чтобы они совпадали с индексами X
    y = y.reset_index(drop=True)
    for i in range(len(X) - seq_length + 1):
        X_seq.append(X[i:i + seq_length])
        y_seq.append(y[i + seq_length - 1])
        
    return np.array(X_seq), np.array(y_seq)
   

In [10]:
 window_size = 200

num_samples = (len(all_datas) // window_size) * window_size
all_datas = all_datas.iloc[:num_samples]  # Обрезаем лишние строки

X = all_datas.drop(columns=['target']).values.reshape(-num_samples, window_size, 3)  # Преобразуем в форму (Batch Size, 800, 3)
y = all_datas['target'].values[:num_samples:window_size]  # Каждое значение 'target' соответствует одному окну из 800

# Проверим итоговые формы X и y
print("Форма X:", X.shape)  # Должно быть (Количество выборок, 800, 3)
print("Форма y:", y.shape)  # Должно быть (Количество выборок,)

# Разделение на обучающую и валидационную выборки
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели
model = Sequential([
    Input(shape=(200, 3)),  # Входной слой с формой (800, 3)
    Bidirectional(LSTM(200, return_sequences=True)),
    Dropout(0.4),
    Flatten(),
    Dense(3, activation='softmax')  # Изменяем на 3, поскольку у нас 3 класса (ds, is, swd)
])

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Вывод структуры модели
model.summary()

# Обучение модели
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Оценка модели
model.evaluate(X_val, y_val)

Форма X: (63070, 200, 3)
Форма y: (63070,)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional)      │ (None, 200, 400)            │         326,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 200, 400)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 80000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │         240,003 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 566,403 (2.16 MB)

 Trainable params: 566,403 (2.16 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 347s 217ms/step - accuracy: 0.8953 - loss: 0.3704 - val_accuracy: 0.9193 - val_loss: 0.2588
Epoch 2/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 352s 223ms/step - accuracy: 0.9196 - loss: 0.2566 - val_accuracy: 0.9201 - val_loss: 0.2511
Epoch 3/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 351s 223ms/step - accuracy: 0.9212 - loss: 0.2512 - val_accuracy: 0.9172 - val_loss: 0.2482
Epoch 4/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 370s 235ms/step - accuracy: 0.9181 - loss: 0.2480 - val_accuracy: 0.9159 - val_loss: 0.2662
Epoch 5/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 370s 235ms/step - accuracy: 0.9209 - loss: 0.2424 - val_accuracy: 0.9213 - val_loss: 0.2430
Epoch 6/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 358s 227ms/step - accuracy: 0.9211 - loss: 0.2405 - val_accuracy: 0.9206 - val_loss: 0.2369
Epoch 7/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 362s 230ms/step - accuracy: 0.9197 - loss: 0.2401 - val_accuracy: 0.9211 - val_loss: 0.2392
Epoch 8/10
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 359s 227ms/step - ac

[0.23366212844848633, 0.9206437468528748]

In [14]:
def get_dataset(name):
    #Загружаем датасет
    def read_dataset(file_path):
        edf_file = pyedflib.EdfReader(file_path)
        n_signals = edf_file.signals_in_file
        signal_labels = edf_file.getSignalLabels()
        signals = [edf_file.readSignal(i) for i in range(n_signals)]
        edf_file.close()
        print('Сигналов обнаружено: ', n_signals)
        return signal_labels, signals

    def read_txt_markers(file_path):
        with open(file_path, 'r') as file:
            labels = file.read().splitlines()
        return labels
    
    def convert_to_sec(time: str):
        s = list(map(int, time.split(':')))
        return s[0]*3600 + s[1] * 60 + s[2]

    def get_markered_dataset(data_file_path, labels_file_path):
        signal_labels, signals = read_dataset(data_file_path)
        signals = np.array(signals)
        data = pd.DataFrame(signals).T.rename(columns={i: signal_labels[i] for i in range(len(signal_labels))})
        
        markers = read_txt_markers(labels_file_path)[1:]
        markers = [line.split('\t') for line in markers]
        markers = [[line[0], convert_to_sec(line[2])*400, line[1]] for line in markers]
        
        markers_df = pd.DataFrame(markers, columns=['id', 'time', 'marker'])
        markers_df['time'] = markers_df['time'].astype(int)
    
        data['target'] = None
    
        for i in range(0, len(markers_df), 2):
            start_marker = markers_df.iloc[i]
            end_marker = markers_df.iloc[i + 1]
            if start_marker['marker'].startswith('ds'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'ds'
            elif start_marker['marker'].startswith('is'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'is'
            elif start_marker['marker'].startswith('swd'):
                mask = list(range(start_marker.time, end_marker.time))
                data.loc[mask, 'target'] = 'swd'
        percentage_marked = data[data['target'].notna()].shape[0]/data.shape[0]
        print(f'Размечено {round(percentage_marked, 2)} данных')
        return data
    
    def load_marked_dataset(file, folder='ECoG_fully_marked_(4+2 files, 6 h each)', base_path=r"Downloads/train_dataset_minzdrav_train"):
        dataset_file_path = fr"{base_path}/{folder}/{file}.edf"
        markers_file_path = fr"{base_path}/{folder}/{file}.txt"
        dataset = get_markered_dataset(dataset_file_path, markers_file_path)
        return dataset

    dataset = load_marked_dataset(name)
    dataset = dataset[dataset['target'].notna()]
    return dataset

In [15]:
test = get_dataset('Ati4x3_9m_Xyl01(Pharm!)_6h')
test

Сигналов обнаружено:  3
Размечено 0.59 данных


,FrL,FrR,OcR,target
26000,0.006125,-0.155688,0.045062,swd
26001,-0.013750,-0.134125,-0.017438,swd
26002,-0.055000,-0.136937,-0.057938,swd
26003,-0.091125,-0.132625,-0.083000,swd
26004,-0.128062,-0.134188,-0.090250,swd
...,...,...,...,...
8639195,-0.059438,-0.052625,-0.074625,swd
8639196,-0.049563,-0.054312,-0.111813,swd
8639197,-0.028438,-0.044062,-0.088938,swd
8639198,-0.049313,-0.049688,-0.061062,swd


In [18]:
test['target'] = test['target'].replace({'ds': 0, 'is': 1, 'swd': 2})
#test1 = test[:800]
X_test = test.drop(columns=['target'])#.values.reshape(-1, 2923600, 3)
y_test = test['target'].values#[:num_samples:window_size]  # Каждое значение 'target' соответствует одному окну из 800 строк
#model.evaluate(X_test, y_test)

In [19]:
# Размер окна и количество каналов, как в обучении
window_size = 200
num_channels = 3

# Убедимся, что количество строк в тестовом наборе делится на размер окна
num_samples = (len(test) // window_size) * window_size
test = test.iloc[:num_samples]  # Обрезаем лишние строки

# Преобразуем тестовые данные в форму (количество окон, 800, 3)
X_test = test.drop(columns=['target']).values.reshape(-1, window_size, num_channels)
y_test = test['target'].values[:num_samples:window_size]  # Берем метки для каждого окна

# Оценка модели на всём тестовом наборе
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Точность на всём тестовом наборе: {test_accuracy * 100:.2f}%")
print(f"Потери (loss) на всём тестовом наборе: {test_loss:.4f}")

799/799 ━━━━━━━━━━━━━━━━━━━━ 42s 52ms/step - accuracy: 0.9660 - loss: 0.1759
Точность на всём тестовом наборе: 92.04%
Потери (loss) на всём тестовом наборе: 0.4202


In [23]:
df = pd.read_csv('valid_file.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [27]:
# Размер окна и количество каналов, как в обучении
window_size = 200
num_channels = 3

# Убедимся, что количество строк в тестовом наборе делится на размер окна
num_samples = (len(df) // window_size) * window_size
df = df.iloc[:num_samples]  # Обрезаем лишние строки

# Преобразуем тестовые данные в форму (количество окон, 800, 3)
X_test = df.drop(columns=['target']).values.reshape(-1, window_size, num_channels)
y_test = df['target'].values[:num_samples:window_size]  # Берем метки для каждого окна

# Оценка модели на всём тестовом наборе
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Точность на всём тестовом наборе: {test_accuracy * 100:.2f}%")
print(f"Потери (loss) на всём тестовом наборе: {test_loss:.4f}")

267/267 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9381 - loss: 0.3498
Точность на всём тестовом наборе: 90.59%
Потери (loss) на всём тестовом наборе: 0.4516
